In [1]:
## import dependencies
import pandas as pd
import requests
import plotly.plotly

In [2]:
## function to fetch 30 days worth of data for a state, 
## subset by the top 10 sites with the most # of species
## return json object with location name, lat longs, & # 
## of species & json object with top 5 most observed birds 
## at each location 
def getStateData(state):

    # Part 1 - Get data
    # set url, token, & request data
    url = f"https://ebird.org/ws2.0/data/obs/US-{state}/recent/?back=30"
    headers = {'X-eBirdApiToken': 'oos4bb9k3art'}
    response = requests.request("GET", url, headers=headers)

    # Part 2 - Get top 10 locations
    # store reponse test
    data = response.text
    # replace true & false
    data = data.replace('true', 'True')
    data = data.replace('false', 'False')
    # get list out of string
    data = eval(data)
    # convert to pandas df
    df = pd.DataFrame(data)

    return(df)

def getTop10(df):
    # create subset of columns to evaluate
    df_s = df.loc[:,['locName', 'lat', 'lng','comName']]
    # remove duplicate records
    df_s = df_s.drop_duplicates()
    # create count of species observations from each location
    counts = df_s.groupby(['locName', 'lat', 'lng'])['locName'].count()
    # sort by the species count
    counts = counts.sort_values(ascending = False)
    # subset for top 10
    top_ten = counts[0:10]
    # rename the series
    top_ten = top_ten.rename('species_number')
    # convert series to dataframe
    top_ten = pd.DataFrame(top_ten)
    # reset index
    top_ten = top_ten.reset_index()
    # convert to json
    top_tenj = top_ten.to_json(orient='records', force_ascii=False)
    top_tenj = top_tenj.replace("\\","")
    return(top_tenj)

    # Part 3 - Get top 5 species at each location
    # subset original df for the top 10 locations
def getBirds(df):
    sp = df.loc[:,['comName', 'howMany', 'locName']]
    # group species to sum the total # of each species 
    # has been recorded at that location
    sp = pd.DataFrame(sp.groupby(['comName'])['howMany'].sum())
    # reset index
    sp = sp.reset_index()
    # sort by location name & howMany
    sp = sp.sort_values(by = ['howMany'], ascending = False)
    # filter for species level identifications
    spFilter = sp.comName.str.contains('sp\\.')
    spFilter = spFilter[spFilter == False]
    spLevel = sp.filter(items = spFilter.index, axis=0)
    # subset for top 5 species at each location
    topsp = spLevel.head(5)
    # convert to json
    topsp_j = topsp.to_json(orient='records', force_ascii=False)
    topsp_j = topsp_j.replace("\\","")
    topsp_j

    return(topsp_j)

In [8]:
df = getStateData("TX")

(524, 4)

(524, 4)

locName                                                                 lat        lng        
(25.952, -97.335)                                                       25.951580  -97.335213     1
(26.152, -97.625)                                                       26.152337  -97.625171     1
(28.8368,-96.5668)                                                      28.836764  -96.566812     1
(35.5415,-101.7299)                                                     35.541515  -101.729877    1
1001–1299 County Road 622, Dayton US-TX (30.0638,-95.0125)              30.063816  -95.012499     1
1100 Folsom Rd, Amarillo US-TX (35.2247,-101.7238)                      35.224687  -101.723764    3
11705–12043 S Highway 36, Brazoria US-TX (28.9916,-95.5241)             28.991597  -95.524121     1
1215 Cave Speings Drive                                                 30.049547  -98.236828     1
1302 Greenhill Drive, Canyon Lake, Texas, US (29.883, -98.27)           29.882888  -98.270391     1
1315 

locName                                                          lat        lng        
South Padre Is.--WBC/Conv. Center/Laguna Madre Trail (LTC 035)   26.140091  -97.174823     19
Salineño Wildlife Preserve (LTC 080)                             26.514858  -99.116206     13
Galveston--Moody Gardens                                         29.274486  -94.854309     11
Columbus, TX  78934, Columbus US-TX (29.8000,-96.5433)           29.800024  -96.543304      9
River Legacy Park                                                32.782845  -97.116317      8
Estero Llano Grande SP WBC (LTC 054)                             26.126030  -97.955917      8
Sarita Rest Area (US77)(LTC 003)                                 27.133129  -97.792740      7
South Padre Is.--Birding and Nature Center                       26.137486  -97.173557      7
Hargill Playa (Hidalgo Co.)                                      26.446927  -98.006430      7
Home                                                             2

locName                                                         lat        lng       
South Padre Is.--WBC/Conv. Center/Laguna Madre Trail (LTC 035)  26.140091  -97.174823    19
Salineño Wildlife Preserve (LTC 080)                            26.514858  -99.116206    13
Galveston--Moody Gardens                                        29.274486  -94.854309    11
Columbus, TX  78934, Columbus US-TX (29.8000,-96.5433)          29.800024  -96.543304     9
River Legacy Park                                               32.782845  -97.116317     8
Estero Llano Grande SP WBC (LTC 054)                            26.126030  -97.955917     8
Sarita Rest Area (US77)(LTC 003)                                27.133129  -97.792740     7
South Padre Is.--Birding and Nature Center                      26.137486  -97.173557     7
Hargill Playa (Hidalgo Co.)                                     26.446927  -98.006430     7
Home                                                            29.657914  -98.664833 

,locName,lat,lng,species_number
0,South Padre Is.--WBC/Conv. Center/Laguna Madre...,26.140091,-97.174823,19
1,Salineño Wildlife Preserve (LTC 080),26.514858,-99.116206,13
2,Galveston--Moody Gardens,29.274486,-94.854309,11
3,"Columbus, TX 78934, Columbus US-TX (29.8000,-...",29.800024,-96.543304,9
4,River Legacy Park,32.782845,-97.116317,8
5,Estero Llano Grande SP WBC (LTC 054),26.126030,-97.955917,8
6,Sarita Rest Area (US77)(LTC 003),27.133129,-97.792740,7
7,South Padre Is.--Birding and Nature Center,26.137486,-97.173557,7
8,Hargill Playa (Hidalgo Co.),26.446927,-98.006430,7
9,Home,29.657914,-98.664833,6


'[{"locName":"South Padre Is.--WBC/Conv. Center/Laguna Madre Trail (LTC 035)","lat":26.1400912,"lng":-97.1748233,"species_number":19},{"locName":"Salineño Wildlife Preserve (LTC 080)","lat":26.5148584,"lng":-99.1162062,"species_number":13},{"locName":"Galveston--Moody Gardens","lat":29.2744864,"lng":-94.8543092,"species_number":11},{"locName":"Columbus, TX  78934, Columbus US-TX (29.8000,-96.5433)","lat":29.800024,"lng":-96.543304,"species_number":9},{"locName":"River Legacy Park","lat":32.7828445,"lng":-97.1163169,"species_number":8},{"locName":"Estero Llano Grande SP WBC (LTC 054)","lat":26.1260304,"lng":-97.9559173,"species_number":8},{"locName":"Sarita Rest Area (US77)(LTC 003)","lat":27.1331291,"lng":-97.7927399,"species_number":7},{"locName":"South Padre Is.--Birding and Nature Center","lat":26.1374859,"lng":-97.1735573,"species_number":7},{"locName":"Hargill Playa (Hidalgo Co.)","lat":26.446927,"lng":-98.00643,"species_number":7},{"locName":"Home","lat":29.6579139,"lng":-98.6648